In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import seaborn as sns
import statsmodels.formula.api as smf
import torch

In [8]:
file = r"C:\Users\PcLaptop\Documents\GitHub\Climate-and-conflict\tpc_prot_dummy.csv"
df_prot = pd.read_csv(file)

file = r"C:\Users\PcLaptop\Documents\GitHub\Climate-and-conflict\tpc_no_prot_dummy.csv"
df_no_prot = pd.read_csv(file)
df_no_prot.dropna(inplace=True)

In [3]:
# noise
df_no_prot['TA_noise'] = df_no_prot['TA'] + np.random.normal(0, 0.1, len(df_no_prot))
df_no_prot['PA_noise'] = df_no_prot['PA'] + np.random.normal(0, 0.1, len(df_no_prot))
df_no_prot['DL_noise'] = df_no_prot['DL'] + np.random.normal(0, 0.1, len(df_no_prot))

df_prot['TA_noise'] = df_prot['TA'] + np.random.normal(0, 0.1, len(df_prot))
df_prot['PA_noise'] = df_prot['PA'] + np.random.normal(0, 0.1, len(df_prot))
df_prot['DL_noise'] = df_prot['DL'] + np.random.normal(0, 0.1, len(df_prot))

In [4]:
y_var_name = 'conflicts'
X_var_names = ['TA','PA','DL']

In [12]:
#from month number to month name
pd.to_datetime(df_no_prot['month'], format='%m').dt.month_name()

0             May
1            June
2            July
3          August
4       September
          ...    
2731       August
2732    September
2733      October
2734     November
2735     December
Name: month, Length: 2736, dtype: object

In [13]:
unit_names = df_no_prot['admin1'].unique().tolist()
unit_names.sort()
unit_names_t = pd.to_datetime(df_no_prot['month'], format='%m').dt.month_name().unique().tolist()
unit_names_mr = (df_no_prot['admin1'] + pd.to_datetime(df_no_prot['month'], format='%m').dt.month_name()).unique().tolist()

lsdv_expr = y_var_name + ' ~ '
i = 0
for X_var_name in X_var_names:
    if i > 0:
        lsdv_expr = lsdv_expr + ' + ' + X_var_name
    else:
        lsdv_expr = lsdv_expr + X_var_name
    i = i + 1
for dummy_name in unit_names[:-1]:
    lsdv_expr = lsdv_expr + ' + ' + dummy_name
for dummy_name_t in unit_names_t[:-1]:
    lsdv_expr = lsdv_expr + ' + ' + dummy_name_t
for dummy_name_mr in unit_names_mr[:-1]:
    lsdv_expr = lsdv_expr + ' + ' + dummy_name_mr
 
print('Regression expression for OLS with dummies=' + lsdv_expr)

Regression expression for OLS with dummies=conflicts ~ TA + PA + DL + Awdal + Bakool + Banadir + Bari + Bay + Galgaduud + Gedo + Hiraan + Lower_Juba + Lower_Shabelle + Middle_Juba + Middle_Shabelle + Mudug + Nugaal + Sanaag + Sool + Togdheer + May + June + July + August + September + October + November + December + January + February + March + AwdalMay + AwdalJune + AwdalJuly + AwdalAugust + AwdalSeptember + AwdalOctober + AwdalNovember + AwdalDecember + AwdalJanuary + AwdalFebruary + AwdalMarch + AwdalApril + BakoolMay + BakoolJune + BakoolJuly + BakoolAugust + BakoolSeptember + BakoolOctober + BakoolNovember + BakoolDecember + BakoolJanuary + BakoolFebruary + BakoolMarch + BakoolApril + BanadirMay + BanadirJune + BanadirJuly + BanadirAugust + BanadirSeptember + BanadirOctober + BanadirNovember + BanadirDecember + BanadirJanuary + BanadirFebruary + BanadirMarch + BanadirApril + BariMay + BariJune + BariJuly + BariAugust + BariSeptember + BariOctober + BariNovember + BariDecember + Bar

In [15]:
lsdv_model = smf.ols(formula=lsdv_expr, data=df_no_prot)
lsdv_model_results = lsdv_model.fit()
print(lsdv_model_results.summary())

                            OLS Regression Results                            
Dep. Variable:              conflicts   R-squared:                       0.307
Model:                            OLS   Adj. R-squared:                  0.246
Method:                 Least Squares   F-statistic:                     5.103
Date:                Tue, 18 Jul 2023   Prob (F-statistic):           1.63e-93
Time:                        15:37:59   Log-Likelihood:                -10449.
No. Observations:                2736   AIC:                         2.134e+04
Df Residuals:                    2517   BIC:                         2.263e+04
Df Model:                         218                                         
Covariance Type:            nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept               